## Data Cleaning

Data below from here: https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-Beginning-February-202/wujg-7c2s

In [13]:
import pandas as pd
df = pd.read_csv("MTA_Ridership_Data.csv")
#The data is from the link above. It is about 2.3GB in size and has over 12 million rows and 12 columns.
#I don't recommend downloading it...

Goal: create average daily, monthly, and average monthly ridership figures for each station, and save the cleaned data as csv files. Then, using the average monthly ridership data, plot the stations onto a scatter_mapbox to create visualizations about the stations' locations and traffic volume, as well as the areas they serve.

In [14]:
#Keep only the columns we need

cols = ['transit_timestamp',
        'station_complex_id',
        'station_complex',
        'borough',
        'routes',
        'ridership',
        'latitude',
        'longitude']
df = df[cols]

#Modify the transit timestamp. Currently the timestamp records hours which is too specific for our purposes
#So just chop everything after the date off
df['transit_timestamp'] = df["transit_timestamp"].str[:10]

#Sum all the monthly ridership values for each station on the same day
df = df.groupby(['transit_timestamp',
                 'station_complex_id',
                 'station_complex',
                 'borough',
                 'routes',
                 'latitude',
                 'longitude'])[["ridership"]].sum()

In [15]:
#I renamed the dataframe above to basically set a checkpoint
#importing the entire raw dataset with read_csv can take a few minutes each time so I decided to sacrifice some memory instead
df1 = df.reset_index()

#rename the ridership column to better reflect what it now represents
df1 = df1.rename(columns={'ridership' : 'daily_ridership'})

#export to csv
df1.to_csv("MTA Daily Ridership By Station.csv")
df1

,transit_timestamp,station_complex_id,station_complex,borough,routes,latitude,longitude,daily_ridership
0,01/01/2023,A002,"Lexington Av (N,R,W)/59 St (4,5,6)",M,"5,N,R,4,,W,6",40.762661,-73.967255,13661
1,01/01/2023,A006,"5 Av-59 St (N,R,W)",M,"N,R,W",40.764812,-73.973351,8910
2,01/01/2023,A010,"57 St-7 Av (N,Q,R,W)",M,"N,R,Q,W",40.764664,-73.980659,15523
3,01/01/2023,A013,"49 St (N,R,W)",M,"N,R,W",40.759899,-73.984138,12783
4,01/01/2023,A022,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",M,"F,M,N,R,Q,B,W,D",40.749718,-73.987823,43513
...,...,...,...,...,...,...,...,...
269433,12/31/2022,R728,Gun Hill Rd (5),BX,5,40.869526,-73.846382,1240
269434,12/31/2022,R729,Baychester Av (5),BX,5,40.878662,-73.838593,719
269435,12/31/2022,R730,Eastchester-Dyre Av (5),BX,5,40.888302,-73.830833,871
269436,12/31/2022,S101,St George,SI,SIR,40.643748,-74.073643,1715


In [16]:
#We now want the monthly ridership. Our data ranges from 02/2022-10/2023
#so we need to account for both the month and the year.
#extract these from the timestamps while disregarding the day of the month
df1['date'] = df1["transit_timestamp"].str[6:].astype(str) + "-" + df1['transit_timestamp'].str[:2].astype(str)

#get rid of the timestamp column, as date serves the same purpose for us now
cols = ['date',
        'station_complex_id',
        'station_complex',
        'borough',
        'routes',
        'daily_ridership',
        'latitude',
        'longitude']
df1 = df1[cols]

#sum all the ridership from rows in the same month at the same station
df1 = df1.groupby(['date',
                  'station_complex_id',
                  'station_complex',
                  'borough',
                  'routes',
                  'latitude',
                  'longitude'])[['daily_ridership']].sum()

In [17]:
#another checkpoint, same reasons as above
df2 = df1.reset_index()

#fix new column name
df2 = df2.rename(columns={'daily_ridership' : 'monthly_ridership'})

#export to csv
df2.to_csv("MTA Monthly Ridership By Station.csv")
df2

,date,station_complex_id,station_complex,borough,routes,latitude,longitude,monthly_ridership
0,2022-02,A002,"Lexington Av (N,R,W)/59 St (4,5,6)",M,"5,N,R,4,,W,6",40.762661,-73.967255,605659
1,2022-02,A006,"5 Av-59 St (N,R,W)",M,"N,R,W",40.764812,-73.973351,166552
2,2022-02,A010,"57 St-7 Av (N,Q,R,W)",M,"N,R,Q,W",40.764664,-73.980659,388839
3,2022-02,A013,"49 St (N,R,W)",M,"N,R,W",40.759899,-73.984138,269150
4,2022-02,A022,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",M,"F,M,N,R,Q,B,W,D",40.749718,-73.987823,1280956
...,...,...,...,...,...,...,...,...
8920,2023-10,R728,Gun Hill Rd (5),BX,5,40.869526,-73.846382,68519
8921,2023-10,R729,Baychester Av (5),BX,5,40.878662,-73.838593,42179
8922,2023-10,R730,Eastchester-Dyre Av (5),BX,5,40.888302,-73.830833,59933
8923,2023-10,S101,St George,SI,SIR,40.643748,-74.073643,167662


In [18]:
#we now want average monthly ridership
#02/2022-10/2023 is 21 months of data
#sum up all the monthly ridership for each station and divide by 21 to get average
df2 = df2.groupby(['station_complex_id',
                  'station_complex',
                  'borough',
                  'routes',
                  'latitude',
                  'longitude']).sum(['monthly_ridership'])/21

In [19]:
#checkpoint
df3 = df2.reset_index()

#rename columns
df3 = df3.rename(columns={'latitude' : 'Latitude',
                          'longitude' : 'Longitude',
                          'monthly_ridership' : 'Average Monthly Ridership',
                          'station_complex' : 'Station Complex'})

#export csv
df3.to_csv('MTA Average Monthly Ridership By Station.csv')
df3

,station_complex_id,Station Complex,borough,routes,Latitude,Longitude,Average Monthly Ridership
0,A002,"Lexington Av (N,R,W)/59 St (4,5,6)",M,"5,N,R,4,,W,6",40.762661,-73.967255,8.877167e+05
1,A006,"5 Av-59 St (N,R,W)",M,"N,R,W",40.764812,-73.973351,2.646410e+05
2,A010,"57 St-7 Av (N,Q,R,W)",M,"N,R,Q,W",40.764664,-73.980659,5.837353e+05
3,A013,"49 St (N,R,W)",M,"N,R,W",40.759899,-73.984138,4.177572e+05
4,A022,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",M,"F,M,N,R,Q,B,W,D",40.749718,-73.987823,1.842760e+06
...,...,...,...,...,...,...,...
420,R728,Gun Hill Rd (5),BX,5,40.869526,-73.846382,7.782733e+04
421,R729,Baychester Av (5),BX,5,40.878662,-73.838593,4.599100e+04
422,R730,Eastchester-Dyre Av (5),BX,5,40.888302,-73.830833,6.400490e+04
423,S101,St George,SI,SIR,40.643748,-74.073643,1.521661e+05


## Visualizations

Let's focus on the average monthly ridership data we just cleaned and calculated from above. We will use plotly to plot the station locations in relation to their average monthly ridership, the ridership density in different areas of the city, and the areas within approximately half a mile from a subway station.

The first plot relates the average monthly ridership geographically to the location of each station, helping to give an idea of where the highest ridership stations are clustered.

In [20]:
#now we plot figures with plotly
from plotly import express as px

#station location and avg monthly ridership at each station
fig = px.scatter_mapbox(df3, 
                        lat = "Latitude",
                        lon = "Longitude",
                        color = "Average Monthly Ridership",
                        range_color=[0,4100000],
                        color_continuous_scale=[(0.00, "blue"), (0.05, "purple"), (0.35, "red"), (1.00, "yellow")],
                        hover_name = "Station Complex",
                        zoom = 9.7,
                        height = 800,
                        width = 800,
                        title = "Average Monthly Ridership of the NYC Subway, by Station",
                        mapbox_style = "carto-positron")

fig.show()

Next, the ridership density. This gives a better visual perspective of which areas of the system experience the highest traffic and therefore, presumably, would experience the largest travel disruption should a delay occur in that area.

In [39]:
#ridership density of stations served by the metro
fig = px.density_mapbox(df3,
                        lat = "Latitude",
                        lon = "Longitude",
                        z = "Average Monthly Ridership",
                        range_color=[0,4100000],
                        color_continuous_scale=[(0.00, "blue"), (0.05, "purple"), (0.35, "red"), (1.00, "yellow")],
                        radius=20, 
                        hover_name = "Station Complex",
                        zoom = 9.7,
                        height = 800,
                        width = 800,
                        title = "Average Monthly Ridership Density in Areas Served by NYC Subway Lines",
                        mapbox_style = "carto-positron")
fig.show()

Finally, this plot looks at the accessibility and geographic areas covered by the system.

In [38]:
#areas within a set distance (1/2 miles) from a subway station
#unfortunately plotly doesn't allow for us to fix the sizes of the points when zooming
#in or out, so the best we could do is approximate the default view to about 1/2 mile

fig = px.density_mapbox(df3,
                        lat='Latitude',
                        lon='Longitude',
                        radius=15, 
                        hover_name = "Station Complex",
                        zoom = 9.7,
                        height = 800,
                        width = 800,
                        opacity = 0.3,
                        title = "Areas of New York City Within a ~1/2 Mile Radius of a Subway Station; Yellow = Closest",
                        mapbox_style = "carto-positron")

fig.show()